In [1]:
#with this code, the goal is to predict the distribution of components within a mixture that is in liquid-liquid equilibrium
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import least_squares
from thermo.unifac import UNIFAC, UFSG, UFIP
from thermo.chemical import Chemical
from ugropy import Groups
import phasepy
from phasepy import mixture, component, virialgamma,unifac
from phasepy.equilibrium import lle
from thermo.unifac import UNIFAC_group_assignment_DDBST


Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19


In [2]:
#first we define the system conditions in which we are operating
T=25+273.15   #temperature in Kelvin
P=1.01325     #pressure in bar
#next, we need to define the components in our experiments
components=['acetic acid','hydrogen peroxide','water','diisopropyl ether']
components_initial_values=[[2.6123,'mass'],[4.125,'mass'],[12.375,'mass'],[17.8883,'mass']]  #list of values of each component, which can be given in 'mass' or 'moles'


In [3]:
def get_molar_fractions_zi(components,components_initial_values,T,P):
    components_moles=[]
    for i in range(len(components_initial_values)):
        if components_initial_values[i][1]=='moles':
            components_moles.append(components_initial_values[i][0])
        elif components_initial_values[i][1]=='mass':
            component=Chemical(components[i],T=T,P=P)
            component_molar_mass=component.MW
            components_moles.append(components_initial_values[i][0]/component_molar_mass)
        else:
            raise ValueError("invalid unit for component initial values. Use 'mass' or 'moles'")
    total_moles=sum(components_moles)
    molar_fraction_zi=[moles/total_moles for moles in components_moles]
    return molar_fraction_zi

In [4]:
def get_components_properties(components,T,P):
    #this function will return a list containing the properties of each component that are needed to define a component using phasepy
    components_properties=[]
    for i in range(len(components)):
        component=Chemical(components[i],T=T,P=P)
        properties=[component.Tc,component.Pc/100000,component.Zc,component.Vc*1000000,component.omega]
        component_name=components[i]
        component_subgroup_names=Groups(component_name)
        sub = component_subgroup_names.unifac.subgroups
        properties.append(sub)
        components_properties.append(properties)
    return components_properties



In [5]:
#Now that we have the functions to facilitate obtaining the properties needed to define the liquid-liquid equilibrium, we can start defining our system
def set_components(components,components_properties):
    mixtures_to_use=[]
    for i in range(len(components)):
        component_to_add=component(name=components[i],Tc=components_properties[i][0],Pc=components_properties[i][1],Zc=components_properties[i][2],Vc=components_properties[i][3],w=components_properties[i][4],GC=components_properties[i][5])
        mixtures_to_use.append(component_to_add)
    return mixtures_to_use

In [6]:
#Now we will define our mixture
experiment_molar_fractions=get_molar_fractions_zi(components,components_initial_values,T,P)
component_properties=get_components_properties(components,T,P)
component_properties[1][5]={'OH':2}
mixture_to_use=set_components(components,component_properties)
#now we can define the mixture using phasepy
mix=mixture(mixture_to_use[0],mixture_to_use[1])
for i in range(2,len(mixture_to_use)):
    mix.add_component(mixture_to_use[i])
print(experiment_molar_fractions)
print(component_properties)
print(mixture_to_use)



[0.04236674787127407, 0.11811004303736015, 0.6690113035215717, 0.17051190556979423]
[[590.7, 57.8, 0.2012439313985603, 171.0, 0.4218, {'CH3': 1, 'COOH': 1}], [728.0, 220.0, 0.28240874135993943, 77.7, 0.3582, {'OH': 2}], [647.096, 220.64, 0.22943845208106295, 55.948037267100005, 0.3443, {'H2O': 1}], [500.3, 28.32, 0.2627942543667121, 386.0, 0.34, {'CH3': 4, 'CH': 1, 'CHO': 1}]]
[<phasepy.mixtures.component object at 0x0000017B4E737510>, <phasepy.mixtures.component object at 0x0000017B4B61F010>, <phasepy.mixtures.component object at 0x0000017B50513F10>, <phasepy.mixtures.component object at 0x0000017B56C5C6D0>]


In [7]:
#now we define the unifac model to be used
mix.original_unifac()
eos=virialgamma(mix, actmodel='original_unifac')
T=T
P=P
Z=np.array(experiment_molar_fractions)
x0=np.array([0.02,0.1,0.85,0.03])
w0=np.array([0.05,0.2,0.35,0.4])
lle(x0,w0,Z,T,P,eos,full_output=True)



           T: 298.15
           P: 1.01325
 error_outer: np.float64(8.928194495626168e-09)
 error_inner: np.float64(1.5415675458302733e-10)
        iter: 15
        beta: array([0.8029538, 0.1970462])
       tetha: array([0.])
           X: array([[0.03648281, 0.14503408, 0.81393171, 0.0045514 ],
       [0.0663435 , 0.0083959 , 0.07846759, 0.846793  ]])
           v: [None, None]
      states: ['L', 'L']

In [8]:
def molar_fraction_to_mass_fraction(components,molar_fractions,T,P):
    #this function assumes a calculus base of 1 mole of the mixture
    components_mass=[]
    for i in range(len(components)):
        component=Chemical(components[i],T=T,P=P)
        component_molar_mass=component.MW
        component_mass=molar_fractions[i]*component_molar_mass
        components_mass.append(component_mass)
    total_mass=sum(components_mass)
    mass_fractions=[mass/total_mass for mass in components_mass]
    return mass_fractions
        

In [9]:
aquous_phase=molar_fraction_to_mass_fraction(components,[0.03648281, 0.14503408, 0.81393171, 0.0045514],T,P)
organic_phase=molar_fraction_to_mass_fraction(components,[0.0663435 , 0.0083959 , 0.07846759, 0.846793],T,P)
print(aquous_phase)
print(organic_phase)

[0.09845519813479246, 0.2216969081999439, 0.6589495525424633, 0.020898341122800174]
[0.04320909796149819, 0.003097300059743869, 0.015331369996393883, 0.938362231982364]
